# Отдельные технологии

In [1]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from scipy import stats
from IPython.display import Image
from sklearn.datasets import load_iris, load_boston
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import SVC, NuSVC, LinearSVC, OneClassSVM, SVR, NuSVR, LinearSVR
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")

## 1) Индикатор прогресса

Индикатор прогресса полезен в том случае, когда какой-либо процесс выполняется долго. В этом случае нужно не вызывать одну стандартную команду библиотеки, а разделить операции на базовые и вызывать их в цикле с использованием индикатора прогресса.

Сейчас наиболее часто используется библиотека [tqdm.](https://tqdm.github.io/)

In [2]:
from tqdm import tqdm

In [3]:
for i in tqdm(range(100)):
    pass

100%|██████████| 100/100 [00:00<?, ?it/s]


### Пример с созданием новых признаков

In [4]:
iris = load_iris()
df_iris = pd.DataFrame(iris.data,columns=iris.feature_names)
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


#### 1) Создание нового признака на основе одного признака

In [5]:
df_iris['sepal_length_meters'] = df_iris['sepal length (cm)'] * 0.01
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),sepal_length_meters
0,5.1,3.5,1.4,0.2,0.051
1,4.9,3.0,1.4,0.2,0.049
2,4.7,3.2,1.3,0.2,0.047
3,4.6,3.1,1.5,0.2,0.046
4,5.0,3.6,1.4,0.2,0.050


#### 2) Создание нового признака на основе нескольких признаков

In [6]:
# Вариант 1

# Если существует возможность записать вычисление в векторизованном виде
df_iris['new_feature_1'] = df_iris['sepal length (cm)'] / df_iris['petal width (cm)']
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),sepal_length_meters,new_feature_1
0,5.1,3.5,1.4,0.2,0.051,25.5
1,4.9,3.0,1.4,0.2,0.049,24.5
2,4.7,3.2,1.3,0.2,0.047,23.5
3,4.6,3.1,1.5,0.2,0.046,23.0
4,5.0,3.6,1.4,0.2,0.050,25.0


In [7]:
# Вариант 2

def make_new_feature(x):
    """
    x - одна строка (запись) датасета
    """
    result = x['sepal length (cm)'] / x['petal width (cm)']
    return result

In [8]:
# Метод apply выполняет вычисление поэлементно, 
# axis=1 означает что осуществляется перебор по строкам
df_iris['new_feature_2'] = df_iris.apply(lambda x: make_new_feature(x), axis=1)
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),sepal_length_meters,new_feature_1,new_feature_2
0,5.1,3.5,1.4,0.2,0.051,25.5,25.5
1,4.9,3.0,1.4,0.2,0.049,24.5,24.5
2,4.7,3.2,1.3,0.2,0.047,23.5,23.5
3,4.6,3.1,1.5,0.2,0.046,23.0,23.0
4,5.0,3.6,1.4,0.2,0.050,25.0,25.0


#### 3) Создание нового признака с использованием индикатора прогресса

In [9]:
# индекс датасета
df_iris.index

RangeIndex(start=0, stop=150, step=1)

In [10]:
np.array(df_iris.index)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149], dtype=int64)

In [11]:
df_iris['new_feature_3'] = 0.0
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),sepal_length_meters,new_feature_1,new_feature_2,new_feature_3
0,5.1,3.5,1.4,0.2,0.051,25.5,25.5,0.0
1,4.9,3.0,1.4,0.2,0.049,24.5,24.5,0.0
2,4.7,3.2,1.3,0.2,0.047,23.5,23.5,0.0
3,4.6,3.1,1.5,0.2,0.046,23.0,23.0,0.0
4,5.0,3.6,1.4,0.2,0.050,25.0,25.0,0.0


In [12]:
def make_new_feature_with_long_delay(sepal_length, petal_width):
    result = sepal_length / petal_width
    return result

In [13]:
for ind in tqdm(df_iris.index):
    # с помощью at можно прочитать или записать ячейку данных по индексу и имени колонки
    sepal_length = df_iris.at[ind, 'sepal length (cm)']
    petal_width = df_iris.at[ind, 'petal width (cm)']
    new_value = make_new_feature_with_long_delay(sepal_length, petal_width)
    df_iris.at[ind, 'new_feature_3'] = new_value

100%|██████████| 150/150 [00:00<?, ?it/s]


In [14]:
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),sepal_length_meters,new_feature_1,new_feature_2,new_feature_3
0,5.1,3.5,1.4,0.2,0.051,25.5,25.5,25.5
1,4.9,3.0,1.4,0.2,0.049,24.5,24.5,24.5
2,4.7,3.2,1.3,0.2,0.047,23.5,23.5,23.5
3,4.6,3.1,1.5,0.2,0.046,23.0,23.0,23.0
4,5.0,3.6,1.4,0.2,0.050,25.0,25.0,25.0


## 2) Вывод прогресса при подборе гиперпараметров

К сожалению, вывод индикатора прогресса при использовании метода [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) не предусмотрен.

Но возможно увеличение значения параметра verbose для отображения текущей информации о подбираемых гиперпараметрах.

Использование индикатора прогресса возможно при использовании библиотеки [pactools](https://pactools.github.io/auto_examples/plot_grid_search.html) однако она относительно сложна в настройке.

In [15]:
n_range = np.array(range(5,55,5))
tuned_parameters = [{'n_neighbors': n_range}]
tuned_parameters

[{'n_neighbors': array([ 5, 10, 15, 20, 25, 30, 35, 40, 45, 50])}]

In [16]:
clf_gs = GridSearchCV(KNeighborsClassifier(), tuned_parameters, cv=3, scoring='accuracy', verbose=5)
clf_gs.fit(iris.data, iris.target)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] n_neighbors=5 ...................................................
[CV] .......... n_neighbors=5, score=0.9803921568627451, total=   0.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] n_neighbors=5 ...................................................
[CV] .......... n_neighbors=5, score=0.9803921568627451, total=   0.0s


...

[CV] n_neighbors=10 ..................................................
[CV] ......... n_neighbors=10, score=0.9607843137254902, total=   0.0s


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


[CV] n_neighbors=10 ..................................................
[CV] ......... n_neighbors=10, score=0.9803921568627451, total=   0.0s
[CV] n_neighbors=10 ..................................................
[CV] ......... n_neighbors=10, score=0.9791666666666666, total=   0.0s
[CV] n_neighbors=15 ..................................................
[CV] ......... n_neighbors=15, score=0.9607843137254902, total=   0.0s
[CV] n_neighbors=15 ..................................................
[CV] ......... n_neighbors=15, score=0.9411764705882353, total=   0.0s
[CV] n_neighbors=15 ..................................................
[CV] ........................ n_neighbors=15, score=1.0, total=   0.0s
[CV] n_neighbors=20 ..................................................
[CV] ......... n_neighbors=20, score=0.9607843137254902, total=   0.0s
[CV] n_neighbors=20 ..................................................
[CV] ......... n_neighbors=20, score=0.9019607843137255, total=   0.0s
[CV] n

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.2s finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_neighbors': array([ 5, 10, 15, 20, 25, 30, 35, 40, 45, 50])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=5)

## 3) [Automated machine learning - AutoML](https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D1%82%D0%BE%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BC%D0%B0%D1%88%D0%B8%D0%BD)

#### Определение из Википедии:

Автоматическое обучение машин (AutoML) — это процесс автоматизации сквозного процесса применения обучения машины к задачам реального мира. В типичном приложении обучения машины практический работник должен применить подходящие методы предварительной обработки данных, конструирования признаков, выделения признаков и выбора признаков, которые делают набор данных пригодным для обучения машин. После этих шагов работник должен осуществить выбор алгоритма и оптимизацию гиперпараметров для максимизации прогнозируемой производительности конечной модели обучения машин. Поскольку многие из этих шагов не могут осуществить люди, не будучи экспертами, был предложен AutoML как основанное на искусственном интеллекте решение для всё возрастающей необходимости применения обучения машин. Автоматизация сквозного процесса применения обучения машин даёт преимущество получения более простых решений, более быстрого создания таких решений и моделей, которые часто превосходят модели, построенные вручную. 

Наиболее часто используется библиотека [TPOT.](https://github.com/EpistasisLab/tpot)

# Дополнительные источники

- [Оптимизация гиперпараметров](https://ru.wikipedia.org/wiki/%D0%9E%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%B3%D0%B8%D0%BF%D0%B5%D1%80%D0%BF%D0%B0%D1%80%D0%B0%D0%BC%D0%B5%D1%82%D1%80%D0%BE%D0%B2)

- [Обзор алгоритмов для оптимизации гиперпараметров](https://towardsdatascience.com/algorithms-for-hyperparameter-optimisation-in-python-edda4bdb167)